# SIGNATURE VERIFICATION

In [23]:
import cv2
import numpy as np

# ESCALAMIENTO
# Metodo que permite escalar la imagen a un tamaño por default(100)
# para que todas las imagenes posean el mismo tamaño
def scale(image, height=300):
    # Obtenemos las filas y columnas
    original_h, original_w = np.float32(image.shape[:2])
    # Calculamos el ancho que tendra con respecto al alto(100)
    width = int( height * ( original_w / original_h ) )
    # Escalamos la imagen
    image_scaled = cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA)
    # Retornamos la imagen escalada
    return image_scaled

image = cv2.imread('images/signature.jpg')
cv2.imshow('1.Signature Original', image)
cv2.waitKey(0)
# A ESCALA DE GRISES
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
cv2.imshow('2.Signature GrayScale', image)
cv2.waitKey(0)

# ESCALAMIENTO
image = scale(image)
cv2.imshow('3.Signature Scaled', image)
cv2.waitKey(0)

# ELIMINACION DE RUIDO
image = cv2.fastNlMeansDenoising(image, None, 6, 7, 21)
cv2.imshow('4.Signature Denoised', image)
cv2.waitKey(0)

# BINARIZACION <127 -> 0 (black), >=127 -> 255 (white)
ret, image = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
cv2.imshow('5. Threshold Binary', image)
cv2.waitKey(0)

# THINING
image = thinning(image)
cv2.imshow('5. Thinning', image)
cv2.waitKey(0)


# *** ROTACION ***

# *** EROSION ***
kernel = np.ones((5,5), np.uint8)
image = cv2.dilate(image, kernel, iterations = 1)
cv2.imshow("7. Erosion", image)
cv2.waitKey(0)

# *** DILATACION ***
kernel = np.ones((5,5), np.uint8)
image = cv2.erode(image, kernel, iterations = 1)
cv2.imshow("6. Dilation", image)
cv2.waitKey(0)



cv2.destroyAllWindows()

In [6]:
ret

127.0

In [2]:
from scipy import weave
import numpy as np
import cv2
import sys

def _thinningIteration(im, iter):
	I, M = im, np.zeros(im.shape, np.uint8)
	expr = """
	for (int i = 1; i < NI[0]-1; i++) {
		for (int j = 1; j < NI[1]-1; j++) {
			int p2 = I2(i-1, j);
			int p3 = I2(i-1, j+1);
			int p4 = I2(i, j+1);
			int p5 = I2(i+1, j+1);
			int p6 = I2(i+1, j);
			int p7 = I2(i+1, j-1);
			int p8 = I2(i, j-1);
			int p9 = I2(i-1, j-1);

			int A  = (p2 == 0 && p3 == 1) + (p3 == 0 && p4 == 1) +
			         (p4 == 0 && p5 == 1) + (p5 == 0 && p6 == 1) +
			         (p6 == 0 && p7 == 1) + (p7 == 0 && p8 == 1) +
			         (p8 == 0 && p9 == 1) + (p9 == 0 && p2 == 1);
			int B  = p2 + p3 + p4 + p5 + p6 + p7 + p8 + p9;
			int m1 = iter == 0 ? (p2 * p4 * p6) : (p2 * p4 * p8);
			int m2 = iter == 0 ? (p4 * p6 * p8) : (p2 * p6 * p8);

			if (A == 1 && B >= 2 && B <= 6 && m1 == 0 && m2 == 0) {
				M2(i,j) = 1;
			}
		}
	} 
	"""

	weave.inline(expr, ["I", "iter", "M"])
	return (I & ~M)


def thinning(src):
	dst = src.copy() / 255
	prev = np.zeros(src.shape[:2], np.uint8)
	diff = None

	while True:
		dst = _thinningIteration(dst, 0)
		dst = _thinningIteration(dst, 1)
		diff = np.absolute(dst - prev)
		prev = dst.copy()
		if np.sum(diff) == 0:
			break

	return dst * 255

